# Configuration Azure ML

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

## 1. Infos

In [1]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [2]:
import datetime
maintenant = datetime.datetime.now()
print('Date :', maintenant)

Date : 2020-06-02 14:34:11.428898


In [3]:
from azureml.core import Workspace, Environment

## 2. Installation librairie AzureML service

Note : Le kernel Python 3.6 Azure ML contient déjà Azure ML.


> Release notes Azure ML service : https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes

> Installation : https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py

### 2.1 Pour installer la première fois le SDK

In [ ]:
#pip install azureml-sdk

> Redémarrer ensuite le kernel Jupyter

### 2.2 Pour mettre à jour Azure ML service

In [ ]:
#pip install --upgrade azureml-sdk

Penser à mettre à jour les extras et ensuite à relancer le kernel Jupyter

In [ ]:
#pip install --upgrade azureml-sdk[automl,notebooks,explain,accel-models,services,tensorboard]

## 3. Version Azure ML installée

In [4]:
import azureml.core
print("Version Azure ML =",azureml.core.VERSION)

Version Azure ML = 1.6.0


In [ ]:
pip list | grep azureml

> Versions Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/azure-machine-learning-release-notes

## 4. Workspace Azure ML

In [5]:
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "dbd82622-13e7-42bc-9061-a17f74627285") #ID soubscription Azure
resource_group = os.environ.get("RESOURCE_GROUP", "rg_amldemo") #Resource group
workspace_name = os.environ.get("WORKSPACE_NAME", "amldemo") #Nom workspace

In [6]:
from azureml.core import Workspace

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print("Le workspace Azure ML service a été trouvé : OK")
except:
   print("Le workspace Azure ML service n'a pas été trouvé")

Le workspace Azure ML service a été trouvé : OK


In [7]:
# Lecture du fichier de config
ws = Workspace.from_config()
print('Workspace : ' + ws.name, 
      'Région : ' + ws.location, 
      'Groupe de ressources : ' + ws.resource_group, sep='\n')

Workspace : amldemo
Région : westeurope
Groupe de ressources : rg_amldemo


In [8]:
# Infos
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/dbd82622-13e7-42bc-9061-a17f74627285/resourceGroups/rg_amldemo/providers/Microsoft.MachineLearningServices/workspaces/amldemo',
 'name': 'amldemo',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Enterprise',
 'workspaceid': 'cdc39fa8-83ab-4307-b933-6dd0cab441d4',
 'description': '',
 'friendlyName': '',
 'creationTime': '2020-05-07T14:07:29.8718329+00:00',
 'keyVault': '/subscriptions/dbd82622-13e7-42bc-9061-a17f74627285/resourcegroups/rg_amldemo/providers/microsoft.keyvault/vaults/amldemo6235215419',
 'applicationInsights': '/subscriptions/dbd82622-13e7-42bc-9061-a17f74627285/resourcegroups/rg_amldemo/providers/microsoft.insights/components/amldemo8673410895',
 'identityPrincipalId': '5b456316-0b5b-40fa-9b8a-5ecce88e9be0',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/dbd82622-13e7-42bc-9061-a17f74627285/resourcegroups/r

In [9]:
from azureml.core import ComputeTarget, Datastore, Dataset

print("- Compute Targets :")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ':', compute.type)
    
print("- Datastores :")
for datastore_name in ws.datastores:
    datastore = Datastore.get(ws, datastore_name)
    print("\t", datastore.name, ':', datastore.datastore_type)
    
print("- Datasets :")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name)

- Compute Targets :
	 compinst01 : ComputeInstance
	 clus01 : AmlCompute
	 compinst02 : ComputeInstance
	 clus02 : AmlCompute
- Datastores :
	 azureml_globaldatasets : AzureBlob
	 workspacefilestore : AzureFile
	 workspaceblobstore : AzureBlob
- Datasets :
	 diabetes_dstore


# Connect Azure Storage

Creer un Azure Datalake Gen 2 dans un groupe de resource

How to:https://docs.microsoft.com/en-us/azure/storage/common/storage-account-create?toc=%2Fazure%2Fstorage%2Fblobs%2Ftoc.json&tabs=azure-portal#create-a-storage-account

Ajouter un fileshare


## Fileshare

In [10]:
# example for fileshare registration
#

file_datastore_name='azfilesharesdk' # Name of the datastore to workspace
file_share_name=os.getenv("FILE_SHARE_CONTAINER", "fileshare01") # Name of Azure file share container
account_name=os.getenv("FILE_SHARE_ACCOUNTNAME", "mdlegoadls") # Storage account name
account_key=os.getenv("FILE_SHARE_ACCOUNT_KEY", "EKgaDNKjsZtNN4mL1N08UT8PErOTyHbg57wRDUmlRC1Ot5bwm0on9X7BXquorVEX61zjMRsNPr4/5uWfy9MfpA==") # Storage account access key

file_datastore = Datastore.register_azure_file_share(workspace=ws,
                                                     datastore_name=file_datastore_name, 
                                                     file_share_name=file_share_name, 
                                                     account_name=account_name,
                                                     account_key=account_key)

In [13]:
print("- Datastores :")
for datastore_name in ws.datastores:
    datastore = Datastore.get(ws, datastore_name)
    print("\t", datastore.name, ':', datastore.datastore_type)

- Datastores :
	 azfilesharesdk : AzureFile
	 azureml_globaldatasets : AzureBlob
	 workspacefilestore : AzureFile
	 workspaceblobstore : AzureBlob


In [16]:
from azureml.core import Dataset
   dataset = Dataset.Tabular.from_delimited_files(path = [(azfilesharesdk, 'lego/parts.csv')])

   # preview the first 3 rows of the dataset
   dataset.take(3).to_pandas_dataframe()

IndentationError: unexpected indent (<ipython-input-16-ddf2eb0fe5cb>, line 2)

## Azure Datalake Store Gen 2

In [ ]:
# Create a service principal for ADLS Gen 2

In [ ]:
# example for Azure Datalake Gen 2

adlsgen2_datastore_name = 'adlsgen2datastore'

subscription_id=os.getenv("ADL_SUBSCRIPTION", "<my_subscription_id>") # subscription id of ADLS account
resource_group=os.getenv("ADL_RESOURCE_GROUP", "<my_resource_group>") # resource group of ADLS account

account_name=os.getenv("ADLSGEN2_ACCOUNTNAME", "<my_account_name>") # ADLS Gen2 account name
tenant_id=os.getenv("ADLSGEN2_TENANT", "<my_tenant_id>") # tenant id of service principal
client_id=os.getenv("ADLSGEN2_CLIENTID", "<my_client_id>") # client id of service principal
client_secret=os.getenv("ADLSGEN2_CLIENT_SECRET", "<my_client_secret>") # the secret of service principal

adlsgen2_datastore = Datastore.register_azure_data_lake_gen2(workspace=ws,
                                                             datastore_name=adlsgen2_datastore_name,
                                                             account_name=account_name, # ADLS Gen2 account name
                                                             filesystem='test', # ADLS Gen2 filesystem
                                                             tenant_id=tenant_id, # tenant id of service principal
                                                             client_id=client_id, # client id of service principal
                                                             client_secret=client_secret) # the secret of service principal

# (Optionnel)

## 5. Gestion des environnements
> Documentation : https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-use-environments

### 5.1 Environnements existants

### Liste des environnements

In [ ]:
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Nom :",env)
        print()
        print("Packages :", envs[env].python.conda_dependencies.serialize_to_string())

In [ ]:
# Info environnement Azure ML Minimal

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-Minimal")

In [ ]:
# Info environnement Azure ML Minimal

env = Environment.get(workspace=ws, name="AzureML-Minimal")
print("Nom : ",env)
print()
print("packages : ", env.python.conda_dependencies.serialize_to_string())

In [ ]:
# AzureML-AutoML-DNN

env = Environment.get(workspace=ws, name="AzureML-AutoML-DNN")
print("Nom : ",env)
print()
print("Packages : ", env.python.conda_dependencies.serialize_to_string())

### 5.2 Création d'un environnement

In [ ]:
from azureml.core.environment import CondaDependencies

monenvironnement = Environment(name="monenvironnement")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")

In [ ]:
conda_dep.add_pip_package("scikit-learn==0.22.2")
monenvironnement.python.conda_dependencies=conda_dep

In [ ]:
monenvironnement.environment_variables = {"MESSAGE":"Mon environnement scikit"}

In [ ]:
from azureml.core import ScriptRunConfig, Experiment
myexp = Experiment(workspace=ws, name = "Exemple0-Environnement")

### Création fichier python

In [ ]:
%%writefile exemplepython.py

import random
for n in range(10):
  print(random.randint(1,101))

tags = {"type": "test" , "langage" : "Python"}

runconfig = ScriptRunConfig(source_directory=".", script="exemplepython.py")
runconfig.run_config.target = "local"
runconfig.run_config.environment = monenvironnement

run = myexp.submit(config=runconfig, tags=tags)
run.wait_for_completion(show_output=True)

run.get_environment()

In [ ]:
# Enregistrement environnement
monenvironnement.register(workspace=ws)

### Liste des environnements

In [ ]:
# Liste - on voit bien la création de l'environnement
for name,env in ws.environments.items():
    print("Nom {} \t - version {}".format(name,env.version))

In [ ]:
restored_environment = Environment.get(workspace=ws,name="monenvironnement",version="1")

print("Visualisation environnement :")
restored_environment

## 6. Azure ML Studio

https://ml.azure.com/

<img src="https://github.com/retkowsky/images/blob/master/LandingPage.jpg?raw=true">

## 7. Azure ML CLI (Command Line Interface)

>Documentation Azure ML CLI :
https://docs.microsoft.com/en-us/azure/machine-learning/reference-azure-machine-learning-cli

>Prérequis : Azure CLI
Pour installer Azure CLI sur Windows :
https://docs.microsoft.com/en-us/cli/azure/install-azure-cli-windows?view=azure-cli-latest
<br>

**. Installation Azure ML CLI<br>**
az extension add -n azure-cli-ml

**. Check<br>**
az ml -h<br>

az --version

**. Création ressource groupe<br>**
az group create -n mltestRG -l westeurope

**. Création workspace Azure ML<br>**
az ml workspace create -w monworkspace -g mltestRG

**. Attach workspace Azure ML<br>**
az ml folder attach -w monworkspace -g mltestRG

**. Création compute server mycomputesrv<br>**
az ml computetarget create  amlcompute -n mysrv1 --min-nodes 0 --max-nodes 2   -s STANDARD_D2_V2<br>

az ml computetarget create  amlcompute -n mysrv2 --min-nodes 1 --max-nodes 2   -s STANDARD_D3_V2

**. Liste des compute servers<br>**
az ml computetarget list

**. Liste experimentations<br>**
az ml experiment list

# Documentation

- Documentation :<br> https://docs.microsoft.com/en-us/azure/machine-learning/

- Nouveautés versions Azure ML service: <br>
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes?view=azure-ml-py

- What is Azure ML service SDK?<br>
https://docs.microsoft.com/en-us/python/api/overview/azure/ml/intro?view=azure-ml-py

- Azure ML service CLI<br>
https://docs.microsoft.com/en-us/cli/azure/ext/azure-cli-ml/ml?view=azure-cli-latest

- Azure ML Visual Interface<br>
Algorithm & module reference overview<br>
https://docs.microsoft.com/en-us/azure/machine-learning/algorithm-module-reference/module-reference


<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">